# TensorFlow・Kerasのトレーニング♨

## CNN - ちょっと高度なCNN 編

## [目次](TableOfContents.ipynb)
- [環境準備](#環境準備)
  - [インポート](#インポート)
  - [共通関数](#共通関数)
  - [CIFAR-10データ](#CIFAR-10データ)
- [高度なCNN](#高度なCNN)
  - [AlexNet](#AlexNet)
  - [GoogLeNet](#GoogLeNet)
  - [VGG16](#VGG16)
  - [ResNet](#ResNet)

## 参考
開発基盤部会 Wiki
- データマイニング（DM）- Python - DL  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20Python%20-%20DL

## [環境準備](TensorFlowAndKeras0.ipynb)

### インポート

In [ ]:
import io
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
print(tf.__version__)

import keras
print(keras.__version__)
# モデル定義
from keras.models import Model, Sequential, model_from_json, load_model
from keras.layers import Dense, Input, Activation, Flatten, Dropout, LSTM
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from keras.optimizers import SGD, Adam
# その他
from keras.applications.vgg16 import VGG16
from keras.utils import to_categorical
from keras.utils import np_utils

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### 共通関数

#### 画像確認

##### 画像とラベルの確認

In [ ]:
def show_image_info(x, y, label, index):
    print("label: ", label[y[index]])
    print("Image: ")
    plt.imshow(x[index].astype(np.uint8))
    plt.show()

##### ランダムな画像一覧

In [ ]:
def show_image_list(x, y, numOfCls=10, numOfImgInCls=10):
    pos = 1
    plt.figure(figsize=(numOfCls, numOfImgInCls))

    # クラス毎に以下の処理を繰り返す。
    for targetClass in range(numOfCls):
        targetIdx = []
        
        # 当該クラスの画像のインデックスリストを取得
        for i in range(len(y)):
            if y[i] == targetClass:
                targetIdx.append(i)
        
        # 当該クラスのインデックスリストからランダムに選んだ最初のn個の画像を描画
        np.random.shuffle(targetIdx)
        for idx in targetIdx[:numOfImgInCls]:
            plt.subplot(numOfCls, numOfImgInCls, pos)
            plt.imshow(x[idx])
            plt.axis('off')
            pos += 1

plt.show()

##### 誤った推論の画像を表示する関数

In [ ]:
def show_incorrect_image_list(x, y, y_pred, label, numOfImg=10):
    index = (y != y_pred)
    for i, val in enumerate(index):
        if val == True:
            print('predict: ', label[y_pred[i]])
            print('answer : ', label[y[i]])
            show_image_info(x, y, label, i)
            numOfImg -= 1
            if numOfImg <= 0:
                break

#### [分類問題関連](ScikitLearnTraining5.ipynb)

##### 分類問題のメトリック表示関数

In [ ]:
def print_metrics(label, pred):
    print('accuracy: %.3f' % metrics.accuracy_score(label, pred)) # 正答率
    
    print('\nmicro') # ミクロ平均
    print('recall: %.3f' % metrics.recall_score(label, pred, average='micro')) # 再現率
    print('precision: %.3f' % metrics.precision_score(label, pred, average='micro')) # 適合率
    print('f1_score: %.3f' % metrics.f1_score(label, pred, average='micro')) # f値
    
    print('\nmacro') # マクロ平均
    print('recall: %.3f' % metrics.recall_score(label, pred, average='macro')) # 再現率
    print('precision: %.3f' % metrics.precision_score(label, pred, average='macro')) # 適合率
    print('f1_score: %.3f' % metrics.f1_score(label, pred, average='macro')) # f値

##### 混同行列のグラフ化関数

In [ ]:
def plot_cm(confmat, label):
    numOfCls = len(label)
    fig, ax = plt.subplots(figsize=(numOfCls, numOfCls))
    ax.matshow(confmat, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(confmat.shape[0]):
        for j in range(confmat.shape[1]):
            ax.text(x=j, y=i, s=confmat[i, j], va='center', ha='center')
            
    # 軸目盛を打つ場所を決める
    ax.set_xticks(np.arange(len(label)))
    ax.set_yticks(np.arange(len(label)))
    # 軸目盛を設定
    ax.set_xticklabels(label)
    ax.set_yticklabels(label)
    #plt.xticks(np.array(label)) # x軸の目盛りを指定
    #plt.yticks(np.array(label)) # y軸の目盛りを指定
    
    plt.xlabel('y_pred label')
    plt.ylabel('y label')
    plt.show()

#### 学習履歴表示関数

##### 損失

In [ ]:
def plot_history_loss(hist):
    plt.plot(hist.history['loss'],label="loss for training")
    plt.plot(hist.history['val_loss'],label="loss for validation")
    plt.title('model loss')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc='best')
    plt.show()

##### 正解率

In [ ]:
def plot_history_acc(hist):
    plt.plot(hist.history['accuracy'],label="accuracy for training")
    plt.plot(hist.history['val_accuracy'],label="accuracy for validation")
    plt.title('model accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend(loc='best')
    plt.show()

### CIFAR-10データ

In [ ]:
#import ssl
#ssl._create_default_https_context = ssl._create_unverified_context
from keras.datasets import cifar10
(x_train_org, y_train_org), (x_test_org, y_test_org) = cifar10.load_data()
print(x_train_org.shape, x_test_org.shape)
print(y_train_org.shape, y_test_org.shape)

In [ ]:
x_org = np.vstack([x_train_org, x_test_org])
y_org = np.concatenate([y_train_org, y_test_org])
print(x_org.shape, y_org.shape)

In [ ]:
x_std = x_org.astype('f')

In [ ]:
x_std /= 255

In [ ]:
# エンコーディング
yy = np_utils.to_categorical(y_org.flatten(), num_classes=10).astype('i') 
# デコーディング
print((yy.argmax(axis=1) == y_org.flatten()).all())

In [ ]:
x_train = x_std[:50000]
x_test = x_std[50000:]
y_train = yy[:50000]
y_test = yy[50000:]
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)
print(x_train[0].shape) # input_shape

## 高度なCNN

### AlexNet
AlexNet の CIFAR10 の accuracy は 0.60 とそれほど良くない。
- https://yamakatsusan.web.fc2.com/CIFAR10_AlexNet_Keras.html
- https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E7%95%B3%E3%81%BF%E8%BE%BC%E3%81%BF%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%EF%BC%88CNN%EF%BC%89#w1e5ff6b

#### モデリング

##### 関数
厳密には以下が異なる。
- 分割・並列化はしていない。
- LRNではなくBNを使用。

In [ ]:
def AlexNet(input_shape, num_classes=10):
    model = Sequential()

    model.add(Conv2D(96, kernel_size=(11, 11),
                     strides=(4, 4),
                     padding='same',
                     activation='relu',
                     input_shape=input_shape))
    
    model.add(MaxPool2D(pool_size=(3, 3),
                           strides=(2, 2),
                           padding='same'))
    
    model.add(BatchNormalization())
    
    model.add(Conv2D(256, kernel_size=(5, 5),
                     strides=(1, 1),
                     padding='same',
                     activation='relu'))
    
    model.add(MaxPool2D(pool_size=(3, 3),
                           strides=(2, 2),
                           padding='same'))
    
    model.add(BatchNormalization())
    
    model.add(Conv2D(384, kernel_size=(3, 3),
                     strides=(1, 1),
                     padding='same',
                     activation='relu'))
    
    model.add(Conv2D(384, kernel_size=(3, 3),
                     strides=(1, 1),
                     padding='same',
                     activation='relu'))
    
    model.add(Conv2D(256, kernel_size=(3, 3),
                     strides=(1, 1),
                     padding='same',
                     activation='relu'))
    
    model.add(MaxPool2D(pool_size=(3, 3),
                           strides=(2, 2),
                           padding='same'))
    
    model.add(BatchNormalization())

    model.add(Flatten())
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2048, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    # コンパイル
    model.compile(optimizer=SGD(lr=0.01), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

##### コンパイル

In [ ]:
model = AlexNet(x_train[0].shape, 10)
model.summary()

#### 実行

##### 学習

```python
batch_size = 500
n_epoch = 30
hist = model.fit(x_train, y_train,
                 validation_data=(x_test, y_test),
                 batch_size=batch_size,
                 epochs=n_epoch,
                 verbose=1)
```

###### 保存
結構時間がかかるのでモデルを保存しておく。

In [ ]:
#model.save('../work/AlexNet.h5')

###### 復元

In [ ]:
model = load_model('../work/AlexNet.h5')

##### 推論

###### 個別

In [ ]:
index = 99
show_image_info(x_org, y_org.flatten(), ['飛','車','鳥','猫','鹿','犬','蛙','馬','船','貨'], index)

predict = model.predict(x_std[index].reshape(1, 32, 32, 3)).argmax()
answer  = y_org[index]

print('predict: ', predict)
print('answer : ', answer)

if predict == answer:
    print('正解')
else:
    print('不正解')

###### 一括

In [ ]:
yy_pred = model.predict(x_std)

#### 評価
訓練データ92%、テストデータ62%、混ぜて87%と過学習している。

In [ ]:
y_pred = np.array(yy_pred.argmax(axis=-1), dtype=np.uint8)
ret = (y_org.flatten() == y_pred)
(len(np.where(ret==True)[0]) / ret.size)

##### [スコアを表示](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20CRISP-DM#uf759972)

In [ ]:
score = model.evaluate(x_std, yy)
print('loss:', score[0])
print('accuracy:', score[1])

In [ ]:
train_score = model.evaluate(x_train, y_train)
print('Train loss:', train_score[0])
print('Train accuracy:', train_score[1])

In [ ]:
test_score = model.evaluate(x_test, y_test)
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

### VGG16
VGG16 の CIFAR10 の accuracy は 0.84 までに改善し、  
ファインチューニングするパターンでは 0.75 程度にしかならないらしい。
- https://qiita.com/tsekine2/items/e95d1cf704247dd3df7b
- https://qiita.com/kotai2003/items/e85f17d7213cf84e3bcd
- https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E7%95%B3%E3%81%BF%E8%BE%BC%E3%81%BF%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%EF%BC%88CNN%EF%BC%89#b9831538

#### モデリング

##### 関数
- 分割・並列化はしていない。
- Batch Normalizationを使用。

In [ ]:
def VGG16(input_shape, num_classes=10):
    model = Sequential()
    
    model.add(Conv2D(
        filters=64, kernel_size=(3,3), strides=(1,1), padding='same',
        input_shape=input_shape, name='block1_conv1'))
    model.add(BatchNormalization(name='bn1'))
    model.add(Activation('relu'))
    
    model.add(Conv2D(
        filters=64, kernel_size=(3,3), strides=(1,1), padding='same',
        name='block1_conv2'))
    model.add(BatchNormalization(name='bn2'))
    model.add(Activation('relu'))
    
    model.add(MaxPool2D(
        pool_size=(2,2), strides=(2,2), padding='same', name='block1_pool'))
    
    model.add(Conv2D(
        filters=128, kernel_size=(3,3), strides=(1,1), padding='same',
        name='block2_conv1'))
    model.add(BatchNormalization(name='bn3'))
    model.add(Activation('relu'))
    
    model.add(Conv2D(
        filters=128, kernel_size=(3,3), strides=(1,1), padding='same',
        name='block2_conv2'))
    model.add(BatchNormalization(name='bn4'))
    model.add(Activation('relu'))
    
    model.add(MaxPool2D(
        pool_size=(2,2), strides=(2,2), padding='same', name='block2_pool'))
    
    model.add(Conv2D(
        filters=256, kernel_size=(3,3), strides=(1,1), padding='same',
        name='block3_conv1'))
    model.add(BatchNormalization(name='bn5'))
    model.add(Activation('relu'))
    
    model.add(Conv2D(
        filters=256, kernel_size=(3,3), strides=(1,1), padding='same',
        name='block3_conv2'))
    model.add(BatchNormalization(name='bn6'))
    model.add(Activation('relu'))
    
    model.add(Conv2D(
        filters=256, kernel_size=(3,3), strides=(1,1), padding='same',
        name='block3_conv3'))
    model.add(BatchNormalization(name='bn7'))
    model.add(Activation('relu'))
    
    model.add(MaxPool2D(
        pool_size=(2,2), strides=(2,2), padding='same', name='block3_pool'))
    
    model.add(Conv2D(
        filters=512, kernel_size=(3,3), strides=(1,1), padding='same',
        name='block4_conv1'))
    model.add(BatchNormalization(name='bn8'))
    model.add(Activation('relu'))
    
    model.add(Conv2D(
        filters=512, kernel_size=(3,3), strides=(1,1), padding='same',
        name='block4_conv2'))
    model.add(BatchNormalization(name='bn9'))
    model.add(Activation('relu'))
    
    model.add(Conv2D(
        filters=512, kernel_size=(3,3), strides=(1,1), padding='same',
        name='block4_conv3'))
    model.add(BatchNormalization(name='bn10'))
    model.add(Activation('relu'))
    
    model.add(MaxPool2D(
        pool_size=(2,2), strides=(2,2), padding='same', name='block4_pool'))
    
    model.add(Conv2D(
        filters=512, kernel_size=(3,3), strides=(1,1), padding='same', 
        name='block5_conv1'))
    model.add(BatchNormalization(name='bn11'))
    model.add(Activation('relu'))
    
    model.add(Conv2D(
        filters=512, kernel_size=(3,3), strides=(1,1), padding='same',
        name='block5_conv2'))
    model.add(BatchNormalization(name='bn12'))
    model.add(Activation('relu'))
    
    model.add(Conv2D(
        filters=512, kernel_size=(3,3), strides=(1,1), padding='same',
        name='block5_conv3'))
    model.add(BatchNormalization(name='bn13'))
    model.add(Activation('relu'))
    
    model.add(MaxPool2D(
        pool_size=(2,2), strides=(2,2), padding='same', name='block5_pool'))
    
    model.add(Flatten(name='flatten'))
    model.add(Dense(units=4096, activation='relu', name='fc1'))
    model.add(Dense(units=4096, activation='relu', name='fc2'))
    model.add(Dense(units=num_classes, activation='softmax', name='predictions'))

    # コンパイル
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

##### コンパイル

In [ ]:
model = VGG16(x_train[0].shape, 10)
model.summary()

#### 実行

##### 学習

```python
batch_size = 500
n_epoch = 30
hist = model.fit(x_train, y_train,
                 validation_data=(x_test, y_test),
                 batch_size=batch_size,
                 epochs=n_epoch,
                 verbose=1)
```

##### 保存
結構時間がかかるのでモデルを保存しておく。

In [ ]:
#model.save('../work/VGG16.h5')

##### 復元

In [ ]:
model = load_model('../work/VGG16.h5')

##### 推論

###### 個別

In [ ]:
index = 99
show_image_info(x_org, y_org.flatten(), ['飛','車','鳥','猫','鹿','犬','蛙','馬','船','貨'], index)

predict = model.predict(x_std[index].reshape(1, 32, 32, 3)).argmax()
answer  = y_org[index]

print('predict: ', predict)
print('answer : ', answer)

if predict == answer:
    print('正解')
else:
    print('不正解')

###### 一括

In [ ]:
yy_pred = model.predict(x_std)

#### 評価
通常の学習で84%、事前学習＆ファインチューニングで75%と言う話だが、実際やったら訓練データ82%、テストデータ70%、混ぜて80%とパフォーマンスは出ていないが[AlexNet](#AlexNet)の過学習の問題は改善している模様。エポック数を30にしたが、20の方が性能が良いのかも？

In [ ]:
y_pred = np.array(yy_pred.argmax(axis=-1), dtype=np.uint8)
ret = (y_org.flatten() == y_pred)
(len(np.where(ret==True)[0]) / ret.size)

##### [スコアを表示](https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E3%83%87%E3%83%BC%E3%82%BF%E3%83%9E%E3%82%A4%E3%83%8B%E3%83%B3%E3%82%B0%EF%BC%88DM%EF%BC%89-%20CRISP-DM#uf759972)

In [ ]:
score = model.evaluate(x_std, yy)
print('loss:', score[0])
print('accuracy:', score[1])

In [ ]:
train_score = model.evaluate(x_train, y_train)
print('Train loss:', train_score[0])
print('Train accuracy:', train_score[1])

In [ ]:
test_score = model.evaluate(x_test, y_test)
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

### GoogLeNet

In [ ]:
model = GoogLeNet(x_train[0].shape, 10)
model.summary()

##### 保存

In [ ]:
model.save('../work/GoogLeNet.h5')

##### 復元

In [ ]:
model = load_model('../work/GoogLeNet.h5')

### ResNet
ResNet-34 の CIFAR10 の accuracy は 0.95 までいけるらしい。
- https://qiita.com/TrashBoxx/items/2d441e46643f73c0ca19
- https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?%E7%95%B3%E3%81%BF%E8%BE%BC%E3%81%BF%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%EF%BC%88CNN%EF%BC%89#x56d1a9e

In [ ]:
model = ResNet(x_train[0].shape, 10)
model.summary()

##### 保存

In [ ]:
model.save('../work/ResNet.h5')

##### 復元

In [ ]:
model = load_model('../work/ResNet.h5')